In [52]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [53]:
alias prepare source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && yarn prepare-demo > /dev/null

In [38]:
prepare

In [54]:
zkml list_datasets

Available datasets:
[
  '14797455496207951391356508759149962584765968173479481191220882411966396840571'
]


In [55]:
zkml list_bounties --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571'

Available bounties on dataset: 14797455496207951391356508759149962584765968173479481191220882411966396840571
[
  {
    PubKey1: '19517200380561602233013947304080897707587995089006775240619814703482168364893',
    PubKey2: '10758991262778566768807205545860719510109920355369254969648901060785189870847',
    MSEcap: '18406',
    Bounty: '49.0',
    Issuer: '0x1CBd3b2770909D4e10f157cABC84C7264073C9Ec',
    IPFS: 'QmWLRJVL5uViT7h64bdeUM3GKMWP9DSWRggGC8igDuQdHR'
  }
]


Command:

```
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --walletprivatekey ./keys/.private_key --mse 18406 --path ./ipfs_dataset
```

In [56]:
zkml download_dataset --hash '14797455496207951391356508759149962584765968173479481191220882411966396840571' --publickey ./keys/out_public.json --walletprivatekey ./keys/.private_key --mse 18406 --path ./ipfs_dataset

In [57]:
!ls ./ipfs_dataset

X.npy Y.npy


In [58]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

In [59]:
X = np.load('./ipfs_dataset/X.npy')
Y = np.load('./ipfs_dataset/Y.npy')

In [60]:
Y.shape, X.shape

((20, 1), (20, 4))

In [61]:
# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X, Y)

# Making Predictions
pred = lr.predict(X)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.07865899873764468


In [63]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('dataset/X.npy',X)
np.save('dataset/Y.npy',Yt_expected)

In [64]:
shuffle = lambda x: np.random.shuffle(x)
shuffle(W)
shuffle(b)
print(mean_squared_error(y_test, np.matmul(X_test, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)

9.435887869889871


In [68]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = 0.07864
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

In [13]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  23.683279741029036
... quantized  65536
Mean Squared Error simulated:  23.68532014893487414
... quantized  65536
Circuit Outputs:
3880370
Proof took 18.778 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:246:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


In [69]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.0786589987376446
... quantized  12891
Mean Squared Error simulated:  0.07864249526948667579
... quantized  12888
Circuit Outputs:
12888
Proof took 18.409 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10850.6
An unexpected error occurred:

<ref *1> Error: cannot estimate gas; transaction may fail or may require manual gas limit (error={"reason":"processing response error","code":"SERVER_ERROR","body":"{\"jsonrpc\":\"2.0\",\"id\":51,\"error\":{\"code\":-32603,\"message\":\"Error: VM Exception while processing transaction: reverted with reason string 'bounty does not exist'\"}}","error":{"code":-32603},"requestBody":"{\"method\":\"eth_estimateGas\",\"params\":[{\"type\":\"0x2\",\"maxFeePerGas\":\"0x9502f900\",\"maxPriorityFeePerGas\":\"0x9502f900\",\"from\":\"0x1cbd3b2770909d4e10f157cabc84c7264073c9ec\",\"to\":\"0x82e01223d51eb87e16a03e24687edf0f294da6f1\",\"data\":\"0xb3ca56c70000000000000000000000002546bcd3c84621e976d8185a91a922

In [30]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '7740372084585987155484008265620943144342913409804205385745482832325610494802',
    'publickey-2': '13432246653756375231286144066765295473357154911140864565409792325468570091416',
    'MSE-Cap': '18406'
  }
]


In [31]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]
